# Homework 5: Project Report
This homework is very similar to homework 5, except now you've hopefully finished your project! You should fill in each of the sections below based on your results. Note that each section has changed from homework 5, so don't just copy paste, the questions are different!

Only one member of the group should submit this assignment, however, please provide the names of each member of the group here:

**Group Members:**
*   Name
*   Name
*   <del>Shine Lin</del>

## Task 1: Problem Description
* Describe the functionality of your finished product. Try to explain it as if you were writing for a potential customor.

    * Our product features some stable basic functions as well as some experimental functions available for your reference. 
    
    * We have developed real-time swing detection, swing spectrum extraction, ball-hit detection and ball-hit quality scoring. These analytical features give you a basic understanding of your ping-pong match progress. 
    
    * Furthermore, our experimental feature allows you to know how well your hit would be without actually serving and hitting a ball. This feature allows you to practice swinging as you begin your ping-pong journey, while saving you from the laborious work of repeatedly hitting and collecting balls as well as the need to use a ping-pong table.
    
* Did anything about your project goals change from the beginning of the quarter? What caused the change?

    **NEEDS REWORDING**

    * Originally planned to classify good/bad players, classify different swings, + all "good" stuff
    
    * Cause: lack of labels (difficulty to find/instruct players to produce desired data)
    

* If you had more time, what would you do differently?

    **TODO**

## Task 2: Dataset Description
* How did you collect your dataset?

  * <b>Feasibility Test</b> We orginally attached a cell phone to a ping-pong paddle and used an app to collect acceleration and gyroscope data. The tester keeps swinging the paddle as if he were to serve a ball.
  
  * <b>FT - Advanced</b> We continued to use this device configuration and let several testers to intermittently either keep swinging or keep walking.
  
  * <b>Milestone 1-3</b> We collected swing-related data using the Inertial Measurement Unit (MPU-6050) and collected hit-related data using several piezoelectric discs. We asked several testers to repeatedly serve the ball (aka. swing first, then forcibly hit the ball).
  
* What features did your dataset have?

    * Raw input features: IMU acceleration/IMU angular velocity ($F^6$), piezoelectric disk reading which indicates vibration strength ($N^3$)

    * Manually labelled features: Player Quality ($B$), Player Id ($N$)

    * Engineering features: Ball Hitting Event ($N$), Ball Hitting Temporal Dispersity (BHTD)/Ball Hitting Spatial Dispersity (BHSD)/Ball Hitting Strength (BHS) ($F^3\times N$), Top-K Signal In Frequency Domain ($F\times N$), Hit Location.

* What were the labels?

    * Some preprocessed features themselves are used as labels; while other labels are manually annotated. Some tasks do not have labels.
    
    * For swing detection and hit detection, there were no labels. However, technically speaking, to fine-tune the algorithm, we did manually visualized and counted swings and hits. The extra information generated by manually counting swings and hits is implicitly encoded into our model, thus could be treated as labels.
    
    * For hit quality prediction, labels are the aforementioned float-type engineering features including BHTD, BHSD, and BHS.
    


* How many samples did you collect?

   * <b>Feasibility Test</b> At least 5 valid files (tests) were collected, each consisting 20+ swings and hits.
   
   * <b>Milestone 1-3</b> At least 13 valid files (tests) were collected. Around 10 of them consist of 120+ swings and hits, and another 2 of them consists of 0 swings or hits for zero offset deteciton purposes; 6 of them (with valid swings and hits) are well annotated with exact number of swings and hits, and the names of the testers to distinguish different swing/hit patterns among different testers. 


* What is your final opinion of your dataset? If you could do it again, what would you change?

    **TODO**

## Task 3: Model Design
* What type of architecture did you use? Was it the same as you expected? If not, why?

  **TODO** Same w/ HW4 for the "experimental feature"
  
  * Hit Location Detection
    
    We initially tried to solve the intersection of three circles to obtain the location of where the ball hits the paddle. These circles have the same location with the deployed three piezo discs, and their diameters are linear to each reading of the sensors. It did not work too well because 1) a single intersection usually does not exist, 2) strong hits close to one sensor could result in a far-away location that is way off the paddle.
    
    We re-implemented the hit location detection algorithm by introducing "weighted centroid" which calculates the centroid of the 3 piezo discs with weights linear to the sensor readings. This worked well (theoretically) if the hit is inside the triangle formed by the 3 discs; otherwise, it is not able to locate.
    
    We eventually dropped this feature due to the fact that sensor readings themselves are unreliable.
    
  * Swing Detection/Counting
  
    We counted the number of swings in a given series of data (length unlimited if thresholds are given; otherwise a minimum length is required to ensure algorithm quality) by 1) finding the local maximas along each dimension of the input features  2) auto-clustering these maximas with a given maxium clustering temporal window along each dimension, and 3) taking the median of number of clusters of all dimensions.
    
    If a threshold is not given (as is the case when our product "meets" a new player for the first time), we require the player to keep swinging for 10+ seconds. After this, our algorithms learns appropriate thresholds for the player by taking 0.6 times the standard deviation along each dimension. This requires no zero-offseting of any dimension.
    
  * Hit Detection/Counting
  
    A similar mechanism is used for hit detection except that a hard-coded filter is applied to piezo disc readings before they are fed to auto-clustering steps. The maximum clustering window for hit detection is smaller than that used in swing detection.
    
  * Swing Spectrum
   
    Fast Fouirer Transform is used for extracting the spectrum for swings. Some tweaks include: 1) we shifted the data first to align its mean to zero before feeding the data to FFT, 2) we drew a partition window to the temporal data if applicable so that we could handle a mixture of "swings & walks" better, 3) we only take some top 10% signals in the frequency domain to generate the spectrum, and 4) we converted frequency domain to period domain so that spectrum has a reasonable "length" along the x-axis (noises are usually of randomly-high frequency which have an extremely short period).
    
  * Hit Quality Prediction/Regression
  
    Depending on the deployment type of the machine learning task, we need to consider the supported operations and architectures by TF Lite as we design and train the model.

    * <b>TF Version</b>

      Input -> SimpleRNN -> Dense -> BatchNorm -> Dropout -> BatchNorm -> Dropout -> Dense (Output)

    * <b>TF Lite Version</b>

      Input -> Conv1D -> Flatten -> Dense -> BatchNorm -> Dropout -> Dense -> BatchNorm -> Dropout -> Dense (Output)

    Since TF Lite does not support RNN nor ELU activations, the RNN has been replaced by Conv1D and ELU has been replaced by ReLU instead.




* Show the structure of your architecture (graphic here)

    For Hit Quality Prediction/Regression Only:

    <img src="https://i.imgur.com/PnHIH5O.jpg"></img>
    
    Total params: 1,360
    
    Trainable params: 1,312
    
    Non-trainable params: 48
  
  


## Task 4: Training
* What was your final training and test accuracy? How many epochs did it take to reach that?

   Results for the regression after 1024 epochs:
   
   * Training Set - Error
   
      err mean: 0.0496, 25-quant: 0.0184, 50-quant: 0.0338, 75-quant: 0.0659
      
   * Test Set - Error
  
      err mean: 0.1631, 25-quant: 0.0658, 50-quant: 0.1609, 75-quant: 0.2441
  
      Note that actual value roughly range from 0-1, while the predicted value exactly range from 0-1.
    
   * Loss Along Epochs
  
      <img src="https://github.com/gixhswlab1ppm/ppm/blob/master/insights/Figure_34.png?raw=true"/>
  
   * Training Set - Error Visualization (order rearranged)
  
      <img src="https://github.com/gixhswlab1ppm/ppm/blob/master/insights/Figure_40.png?raw=true"/>
  
   * Test Set - Error Visualization 1 (order rearranged)
  
      <img src="https://github.com/gixhswlab1ppm/ppm/blob/master/insights/Figure_42.png?raw=true"/>
  
    * Test Set - Error Visualization 2 (order rearranged)
   
      <img src="https://github.com/gixhswlab1ppm/ppm/blob/master/insights/Figure_41.png?raw=true"/>
  
  
* Did you encounter any unexpected issues during training?

  **TODO** Please refer to HW4 and talk about the model initially "not learning weights but only learning bias"

## Task 5: Deployment

**TODO**

* Describe the method used to fun your model, how did it fit into the rest of your system?
* Are you happy with how the project turned out? Why or why not?
  